<a href="https://colab.research.google.com/github/carlos-alves-one/-BlockChain-Explorer-Project/blob/main/blockchain_explorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Goldsmiths University of London
### MSc. Data Science and Artificial Intelligence
### Module: Blockchain Programming
### Author: Carlos Manuel De Oliveira Alves
### Student: cdeol003
### BlockChain Explorer Project

# Goal No.1

Calculate the transaction ID

## Step 1: Setup
Explain the setup process, including importing necessary libraries (e.g., hashlib for cryptographic hashing) and any initial configurations required.

In [2]:
import hashlib


## Step 2: Define Transaction Details
Outline how to define the transaction details, such as sender and recipient addresses, the amount, the fee, the nonce, and the digital signature. Mention that these details form the basis of the transaction and are crucial for its unique identification.

In [3]:
sender_addr      = "b66eeaa8059bd4d50c760beb93c2eb086ad91853"
recipient_addr   = "b8e6687270bade89f03178d0195dbde30be1287c"
sender_publickey = "3056301006072a8648ce3d020106052b8104000a03420004abb6abed08a9304fedaa394bd578fde6e28a3721ef25da5063fd3ddf"
amount = 5
fee    = 1
nonce  = 1
digital_signature = "39d22d5f4c0b28e15428c1a6c55818ade226a9f15968d3b9ef3d15253770f35e"


## Step 3: Concatenate Transaction Details

Describe how to concatenate all transaction details into a single string. This step is critical as it prepares the data for hashing.

In [4]:
transaction_details = f"{sender_addr}{recipient_addr}{amount}{fee}{nonce}{digital_signature}"


## Step 4: Hash the Details

Discuss applying the SHA-256 hash function to the concatenated string to generate the transaction ID. Emphasize the importance of using a cryptographic hash function for security and uniqueness.

In [5]:
transaction_id = hashlib.sha256(transaction_details.encode()).hexdigest()


## Step 5: Output the Transaction ID

Finally, explain how to output the transaction ID, providing a clear conclusion to the process.

In [6]:
print("The calculated transaction ID is:", transaction_id)


The calculated transaction ID is: 78209e0c320e77cdab51e4e1758b28b5b46f12340399f26e4e32356b47e2c067


# Goal No.2

Verify the following transactions:
- Check the sender’s address,
- Check digital signature
- Check transaction ID

## Transaction No.1

In [ ]:
sender_addr      = "3e6cfd7644cc96050c506d5246ea831998a98412"
recipient_addr   = "263b26367061e7c505b520a192c5c3d079f04808"
sender_publickey = "3056301006072a8648ce3d020106052b8104000a03420004b7348c4c2c 97fc8d0e1645fa868d32ca4de03f0772cb76bec66d535d977177896115445298b10cd378dd66afc6bff24a7827083251dd210303d0699f327a7134"
amount = 5
fee    = 1
nonce  = 1
digital_signature = "39d22d5f4c0b28e15428c1a6c55818ade226a9f15968d3b9ef3d15253770f35e"


### Verify Sender's Address

#### Calculate SHA1 Hash of the Public Key

In [ ]:
# Import the 'hashes' module from the 'cryptography.hazmat.primitives' package
from cryptography.hazmat.primitives import hashes

# Define a function to calculate SHA1 hash of a given public key
def calculate_sha1_hash(public_key):

    # Initialize a SHA1 hash object
    digest = hashes.Hash(hashes.SHA1())

    # Update the hash object with the public key data
    digest.update(public_key)

    # Finalize the hash calculation and return the hash value
    return digest.finalize()
